In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
import datetime as dt
import time
import re

In [2]:
# @NOTE: Replace the path with your actual path to the chromedriver
executable_path = {"executable_path": "chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

In [3]:
# def scrape_all():
# 1.......Get Mars news
marsUrl = "https://mars.nasa.gov/news/"
browser.visit(marsUrl)
    
# scrape page into soup
html = browser.html
marsSoup = bs(html, "html.parser")

# update below variables
listItems = marsSoup.find('div', class_='list_text')
news_title = listItems.find_all('a')[0].text
news_p = marsSoup.find('div', class_='article_teaser_body').text

print(news_title)
print(news_p)

NASA, ULA Launch Mars 2020 Perseverance Rover Mission to Red Planet
The agency's Mars 2020 mission is on its way. It will land at Jezero Crater in about seven months, on Feb. 18, 2021. 


In [4]:
    # 2.......Get featured image
spaceUrl = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(spaceUrl)

    # Find the more info button and click that
    
    # Parse the resulting html with soup
html = browser.html
spaceSoup = bs(html, "html.parser")
    
    # Find the relative image url
carouselItems = spaceSoup.find('div', class_='img')
spaceImgPath = carouselItems.find_all('img')[0]['src']

    # Use the base url to create an absolute url
spaceImgURL = spaceUrl + spaceImgPath

print(spaceImgPath)
print(spaceImgURL)

/spaceimages/images/wallpaper/PIA24032-640x350.jpg
https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars/spaceimages/images/wallpaper/PIA24032-640x350.jpg


In [5]:
    # 3.......Get hemispheres
hemiUrl = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemiUrl)

html = browser.html
hemiSoup = bs(html, "html.parser")

    # Click the link, find the sample anchor, extract the href
hemiresults = hemiSoup.find('div', class_='item')
hemiImgPath = hemiresults.find_all('a')[0]['href']
hemiImgUrl = hemiUrl + hemiImgPath

print(hemiImgPath)
print(hemiImgUrl)

/search/map/Mars/Viking/cerberus_enhanced
https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars/search/map/Mars/Viking/cerberus_enhanced


In [6]:
    # 4.......Get twitter weather
weatherUrl = "https://twitter.com/marswxreport?lang=en"
browser.visit(weatherUrl)

html = browser.html
weatherSoup = bs(html, "html.parser")
    # Pause for 5 seconds to let the Twitter page load before extracting the html
time.sleep(5)

    # First, find a tweet with the data-name `Mars Weather`



In [7]:
# pattern = re.compile(r'sol')
# weather_tweet_text =weatherSoup.find('span', text=pattern).text
# print(weather_tweet_text)

AttributeError: 'NoneType' object has no attribute 'text'

In [17]:
weatherResults = weatherSoup.find('div', class_='css-1dbjc4n')
weatherText = weatherResults.find('span', class_="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0")
pattern = re.compile(r'sol')
latest_weather = weatherResults.find_all('span', text=pattern)
# Next, search within the tweet for the p tag or span tag containing the tweet text
    # As Twitter is frequently making changes the try/except will identify the tweet
    # text using a regular expression pattern that includes the string 'sol' if there
    # is no p tag with a class of 'tweet-text'

#     mars_weather = ""
# print(weatherResults.prettify())
# print(weatherText)
print(latest_weather)

[]


In [14]:
    # 5........Get mars facts
factsUrl = "http://space-facts.com/mars/"
browser.visit(factsUrl)
    #use pd.read_html and df.to_html
factsTable = pd.read_html(factsUrl)    

facts_df = factsTable[0]
facts_df = facts_df.rename(columns={0: "Description", 1: "Value"})
facts_df

htmlTable = facts_df.to_html(table_id="html_tbl_css", justify="left", index=False)
htmlTable

mars_data = {
        "Latest_News_Title": news_title ,
        "Latest_News_Paragraph": news_p,
        "Space_Image": spaceImgURL,
        "Hemi_Image": hemiImgUrl,
        "Weather": latest_weather,
        "Table": htmlTable
}
mars_data
# new_df = df.to_d
#     # Store all scraped data in a dictionary
#     data = {}

#     # Stop webdriver and return data
#     browser.quit()

#     return data
#     # End Function

{'Latest News Title': 'NASA, ULA Launch Mars 2020 Perseverance Rover Mission to Red Planet',
 'Latest News Paragraph': "The agency's Mars 2020 mission is on its way. It will land at Jezero Crater in about seven months, on Feb. 18, 2021. ",
 'Space Image': 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars/spaceimages/images/wallpaper/PIA24032-640x350.jpg',
 'Hemi Image': 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars/search/map/Mars/Viking/cerberus_enhanced',
 'Weather': '',
 'Table': '<table border="1" class="dataframe" id="html_tbl_css">\n  <thead>\n    <tr style="text-align: left;">\n      <th>Description</th>\n      <th>Value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <td>Moo

In [ ]:
if __name__ == "__main__":

    # If running as script, print scraped data
    print(scrape_all())